In [1]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'LinkSoul/Chinese-Llama-2-7b',
	'SM_NUM_GPUS': json.dumps(1)
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
    model_data_download_timeout=3600,
	container_startup_health_check_timeout=3000,
  )
  
# send request
predictor.predict({
	"inputs": """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
            If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

用中文回答，When is the best time to visit Beijing, and do you have any suggestions for me? [/INST]""",
})

------------------------!

[{'generated_text': "[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n            If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\n用中文回答，When is the best time to visit Beijing, and do you have any suggestions for me? [/INST]When is the best time to visit Beijing?\n\nThe best time to visit Beij"}]

In [2]:
import pandas as pd
df = pd.read_csv('data/samples.csv', index_col=0)
dfc = pd.read_csv('data/category.csv')


In [3]:
instruction ="""
Please classify the cateory and sub category based on the product name. 
Product name: {0}
1. Return the results in JSON format with the following keys: "product_name", "category", "sub_category", and "common_labels".
2. The "product_name" field in the JSON should be an exact copy of the given product name.
3. Your task is to classify a product name into one of the following categories: "咖啡類","飲料類","水類","乳品類","豆米漿/植物奶","冰品類","鮮食類","烘焙/甜點類","蛋品類","生鮮食品類","冷凍食品類","保健食品類","點心/零食類","民生食材類","日用百貨類","洗浴清潔/保養類","寵物用品類","戶外休閒用品類","家電類","書本/文教用品類","3C類","服飾鞋包類","精品類","傢俱/家飾類","菸酒類","禮盒類"。 
Please return the category as the "category" field. Do not make up a new category. 
Please note that you should accurately categorize each product based on its name and description, 
using your best judgment to determine which category it belongs to. 
Your response should be specific and accurate, ensuring that each product is correctly classified according to its type.

4. Once you have determined the category of the product, please select the best matching 
subcategory list below and classify the product accordingly. 
This result shall be returned in json "sub_category" field

"咖啡類":["現做咖啡飲品","罐裝/瓶裝咖啡","沖泡式咖啡","濾掛式咖啡","即溶咖啡","咖啡相關食用品"]
"飲料類":["現做茶飲品","茶飲","碳酸/汽水","果汁/果醋飲","運動飲料","養生飲品","能量飲料","特殊風味飲品","沖泡類飲品","現調果汁","其他現調飲品"]
"水類":["礦泉水","氣泡水","機能水"]
"乳品類":["鮮乳類","保久乳","調味乳","優格","優酪乳","發酵乳"]
"豆米漿/植物奶":["豆漿","米漿","燕麥奶","其他植物奶"]
"冰品類":["現做霜淇淋/聖代","冰棒/雪糕","冰淇淋","甜筒","仙草/愛玉","水果冰品","冰塊","現調冰品","其他冰品"]
"鮮食類":["蒸箱食品","地瓜","熱狗","關東煮","米飯類","麵食類","三明治","漢堡","生鮮蔬果/沙拉","小菜/滷味","湯品","水餃/點心","雞胸肉","茶葉蛋/水煮蛋","肉品","其他鮮食商品"]
"烘焙/甜點類":["麵包","蛋糕","中式糕點","其他甜食"]
"蛋品類":["生雞蛋","加工蛋"]
"生鮮食品類":["生鮮蔬菜","生鮮水果","生鮮肉品","生鮮海鮮","豆腐","冷藏調理","速食調理包","其他生鮮食品"]
"冷凍食品類":["冷凍蔬菜","冷凍水果","冷凍肉品","冷凍海鮮","火鍋用料","冷凍水餃/點心類","冷凍微波料理","其他冷凍商品"]
"保健食品類":["保健/養生","美肌/纖體","醫材/藥品"]
"點心/零食類":["餅乾","堅果","肉乾/肉紙","豆乾","魷魚絲","海苔","布丁/果凍","果乾","蔬片乾","即飲甜品","糖果","巧克力","口香糖","其他零食"]
"民生食材類":["料理調味","果醬/抹醬","有機食品","罐頭","各式麵條/拌麵","泡麵","嬰幼兒食品","南北乾貨/雜糧食材","米","油","奶粉","沖泡即食品","其他民生食材"]
"日用百貨類":["民生紙品","洗衣用具","家庭清潔用品","家庭清潔洗劑","衣物清潔","嬰幼兒用品","餐廚用品","汽機車用品類","衛浴用品","晴雨相關用具","其他日用百貨"]
"洗浴清潔/保養類":["女性衛生用品","髮部清潔/保養","身體清潔/保養","口腔保健","彩妝/卸妝","男性用品/保養","美容保養"]
"寵物用品類":["貓食","狗食","其他寵物食品","寵物用品"]
"戶外休閒用品類":["運動健身","戶外露營","行李箱/配件"]
"家電類":["大型家電","廚房家電","生活家電","美容家電","視聽娛樂家電","電池充電相關"]
"書本/文教用品類":["書籍","數位內容","文教用品","報紙"]
"3C類":["手機/通訊/週邊","桌機/筆電/平板","電腦螢幕/硬碟","相機/攝影/DV","週邊/耗材","虛擬/點數商品"]
"服飾鞋包類":["品牌鞋款","流行鞋襪","男女內著","童裝","女裝","男裝","個人服飾配件","運動機能服飾"]
"精品類":["鑽石","珠寶／玉石","精品／飾品","手錶","黃金"]
"傢俱/家飾類":["家飾","寢具","傢俱","床墊","燈具"]
"菸酒類":["菸品","威士忌","清酒","調酒","啤酒","紅白酒","香檳氣泡酒","藥酒","高粱"]
"禮盒類":["禮盒"]

5. assign appropriate labels/tags to the product as "common_labels", such as flavor, brand, key ingredient, and package size 
(if applicable), etc. Please avoid choosing duplicate labels. Your labels shall be no more than four.

The json result is:
"""

item = df.sample(1).item_name.iloc[0]
item

prompt = instruction.format(item)
prompt

predictor.predict({"inputs": prompt})

[{'generated_text': '\nPlease classify the cateory and sub category based on the product name. \nProduct name: HH 鋼化玻璃保護貼系列 realme narzo 50A  \n1. Return the results in JSON format with the following keys: "product_name", "category", "sub_category", and "common_labels".\n2. The "product_name" field in the JSON should be an exact copy of the given product name.\n3. Your task is to classify a product name into one of the following categories: "咖啡類","飲料類","水類","乳品類","豆米漿/植物奶","冰品類","鮮食類","烘焙/甜點類","蛋品類","生鮮食品類","冷凍食品類","保健食品類","點心/零食類","民生食材類","日用百貨類","洗浴清潔/保養類","寵物用品類","戶外休閒用品類","家電類","書本/文教用品類","3C類","服飾鞋包類","精品類","傢俱/家飾類","菸酒類","禮盒類"。 \nPlease return the category as the "category" field. Do not make up a new category. \nPlease note that you should accurately categorize each product based on its name and description, \nusing your best judgment to determine which category it belongs to. \nYour response should be specific and accurate, ensuring that each product is correctly classified acc